In [1]:
!pip install SimpleITK
import os
import glob
import SimpleITK as sitk

def normalize_image_intensity(image):
    """
    Normalize the intensities of a given image to the range [0, 1].
    
    Args:
        image (SimpleITK.Image): Input image.
        
    Returns:
        SimpleITK.Image: Normalized image.
    """
    image_array = sitk.GetArrayFromImage(image)
    min_intensity = image_array.min()
    max_intensity = image_array.max()

    return (image - min_intensity) / (max_intensity - min_intensity)

def resample_image(image, reference_image, interpolator=sitk.sitkLinear):
    """
    Resample the input image in the affine space of the reference image.
    
    Args:
        image (SimpleITK.Image): Input image to be resampled.
        reference_image (SimpleITK.Image): Reference image with desired affine space.
        interpolator (SimpleITK.InterpolatorEnum): Interpolator type (default: sitk.sitkLinear).
        
    Returns:
        SimpleITK.Image: Resampled image.
    """
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(reference_image)
    resampler.SetInterpolator(interpolator)

    return resampler.Execute(image)

def main():
    # Set the working directory to the "data/input" folder
    os.chdir('../data/input')
    
    # Get the list of patient folders
    patient_folders = [folder for folder in os.listdir() if os.path.isdir(folder) and folder.startswith('E')]
    
    for patient_folder in patient_folders:
        input_folder = os.path.abspath(patient_folder)
        
        # Read the reference image (T1w_1mm.nii.gz)
        reference_image_file = os.path.join(patient_folder,"output",'T1w_1mm.nii.gz')
        reference_image = sitk.ReadImage(reference_image_file)
        
        # Process each modality
        for modality in ['T2w_1mm_noalign_mask', 'FA_deformed_mask', 'ADC_deformed_mask']:
            input_image_file = os.path.join(input_folder,"output", f'{modality}.nii.gz')
            image = sitk.ReadImage(input_image_file)

            # Normalize the input image and resample to the reference image space
            normalized_image = normalize_image_intensity(image)
            resampled_image = resample_image(normalized_image, reference_image)

            # Save the output image
            output_image_file = os.path.join(input_folder,"output", f'{modality}_preprocessed.nii.gz')
            sitk.WriteImage(resampled_image, output_image_file)

if __name__ == "__main__":
    main()